In [1]:
import pandas as pd

In [2]:
df_f=pd.read_csv("Fake.csv")

In [3]:
df_f["label"]=0

In [4]:
df_t = pd.read_csv("True.csv")

In [5]:
df_t["label"]=1

In [6]:
df= pd.concat([df_f,df_t],axis=0)
# d.drop(["23481"],inp)

In [7]:
# df.drop([23481],inplace=True)

In [8]:
df.reset_index(drop=True,inplace =True)

In [9]:
df.loc[23482,"title"]

'U.S. military to accept transgender recruits on Monday: Pentagon'

In [10]:
X=df.drop(["label","subject","date"],axis=1)
y= df.label

In [11]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding,Dense , LSTM ,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import  one_hot

In [12]:
# df.reset_index(drop=True,inplace =True)

In [13]:
df.tail()


,title,text,subject,date,label
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
44897,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [22]:
df["text"][0]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [14]:
import re
import nltk 
from nltk.corpus import stopwords

In [15]:
from nltk.stem.porter import PorterStemmer

In [16]:
ps = PorterStemmer()

In [17]:
message = X.copy()

In [18]:
#data preprosecing
corpus=[]
for i in range(len(message)):
    review = re.sub('[^a-zA-Z]',' ',str(message.loc[i,"title"]))
    review=review.lower()
    review = review.split()
    
    
    review= [ps.stem(word) for word in review if word not in stopwords.words("english")]
    review =" ".join(review)
    corpus.append(review)
    

In [19]:
voc_size = 20000

In [20]:
onehot_rep= [one_hot(word,voc_size) for word in  corpus]

In [21]:
sen_max=[]
for j in  df["title"]:
    j=j.split(" ")
    
    sen_max.append((len(j)))
    


In [22]:
sent_len = 45
embedded_docs = pad_sequences(sequences=onehot_rep,
    maxlen=45,
    dtype='int32',
    padding='pre',
    truncating='pre',
    value=0.0)

In [23]:
embed_feature = 100
model=Sequential()
model.add(Embedding(input_dim=20000,
    output_dim=100,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=45))
model.add(LSTM(units=100,
    activation="tanh",
    recurrent_activation="sigmoid",
    use_bias=True,
    kernel_initializer="glorot_uniform",
    recurrent_initializer="orthogonal",
    bias_initializer="zeros",
    unit_forget_bias=True,
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
    go_backwards=False,
    stateful=False,
    time_major=False,
    unroll=False
    ))
model.add(Dense(units=1,
    activation="sigmoid",
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None))
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=["accuracy"]
        )

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 45, 100)           2000000   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 2,080,501
Trainable params: 2,080,501
Non-trainable params: 0
_________________________________________________________________


In [25]:
import numpy as np

In [26]:
X_final = np.array(embedded_docs)
y_final = np.array(y)
print(X_final)
(print(type(embedded_docs)))

[[    0     0     0 ... 19664   653 12076]
 [    0     0     0 ...  1676  6558   816]
 [    0     0     0 ...  3679 18387 13510]
 ...
 [    0     0     0 ... 14293 18478  8122]
 [    0     0     0 ... 14127 12924    87]
 [    0     0     0 ...  6657  1676 19352]]
<class 'numpy.ndarray'>


In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train,X_test , y_train , y_test = train_test_split(X_final,y_final,random_state=3 )

In [29]:
model.fit(X_train,y_train ,validation_data=(X_test,y_test),epochs=3)

Epoch 1/3
1053/1053 [==============================] - 54s 50ms/step - loss: 0.1701 - accuracy: 0.9280 - val_loss: 0.1194 - val_accuracy: 0.9547
Epoch 2/3
1053/1053 [==============================] - 51s 49ms/step - loss: 0.0775 - accuracy: 0.9717 - val_loss: 0.1178 - val_accuracy: 0.9563
Epoch 3/3
1053/1053 [==============================] - 51s 49ms/step - loss: 0.0461 - accuracy: 0.9832 - val_loss: 0.1411 - val_accuracy: 0.9531


## adding dropout

In [91]:
model=Sequential()
model.add(Embedding(input_dim=20000,
    output_dim=100,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=45))
model.add(Dropout(0.3))
model.add(LSTM(units=100,
    activation="tanh",
    recurrent_activation="sigmoid",
    use_bias=True,
    kernel_initializer="glorot_uniform",
    recurrent_initializer="orthogonal",
    bias_initializer="zeros",
    unit_forget_bias=True,
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
    go_backwards=False,
    stateful=False,
    time_major=False,
    unroll=False
    ))
model.add(Dropout(0.3))
model.add(Dense(units=1,
    activation="sigmoid",
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None))
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=["accuracy"]
        )

In [30]:
y_pred = model.predict(X_test)

In [31]:
y_pred>0.5

array([[False],
       [False],
       [False],
       ...,
       [ True],
       [ True],
       [False]])

In [32]:
y_predict = []
for i  in y_pred>0.5:
    if i == [False]:
        y_predict.append(0)
    else:
        y_predict.append(1)
        

In [33]:
y_predict

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,


In [34]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [35]:
confusion_matrix(y_test,y_predict)

array([[5660,  253],
       [ 273, 5039]], dtype=int64)

In [36]:
accuracy_score(y_test,y_predict)

0.9531403118040089